## Deployment Mockup

In [ ]:
import pandas as pd
from datetime import datetime
import requests

### 1. Get station name from user and convert it to station id for model.

In [ ]:
trips_lookup = pd.read_csv("trips_lookup.csv")
print(trips_lookup.shape)
trips_lookup.head()

In [ ]:
station_name_user = "One Kendall Square at Hampshire St / Portland St"

In [ ]:
station_id = trips_lookup[trips_lookup["station name"] == station_name_user]["station id"].tolist()[0]
station_id

### 2. Get date and time from user and convert it to timestamp for model.

In [ ]:
datetime_user = "08/30/2022 17:00"

In [ ]:
timestamp = datetime.strptime(datetime_user, "%m/%d/%Y %H:%M").strftime("%Y-%m-%d %H:%M:%S")
timestamp

### 3. Get predictions from trip start model.

### 4. Get predictions from trip stop model.

### 5. Get real-time station status data.

In [ ]:
all_station_status = requests.get("https://gbfs.bluebikes.com/gbfs/en/station_status.json").json()
all_station_status = all_station_status["data"]["stations"]

In [ ]:
for i in range(len(all_station_status)):
    if all_station_status[i]["station_id"] == str(station_id):
        station_status = all_station_status[i]
        break

In [ ]:
realtime_bikes_available = station_status["num_bikes_available"]
realtime_bikes_available

In [ ]:
realtime_docks_available = station_status["num_docks_available"]
realtime_docks_available

### 6. Combine model's predictions with real-time data.

### 7. Display results to the user.